<a href="https://colab.research.google.com/github/SherinBojappa/prompting/blob/main/GPT_3_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
import openai

In [ ]:
# open ai api key
%cd /content/drive/MyDrive/

In [ ]:
!pip install pandas 

In [ ]:
import pandas as pd

In [ ]:
# get openai api key from dashboard
f = open('open_ai_key.txt', 'r')
key = f.readlines()
f.close()
openai.api_key = key[0].strip('\n')

In [ ]:
# zero shot performance does not give the required label
p = """premise: 'Britain said, Friday, that it has barred cleric, Omar Bakri, from returning to the country from Lebanon, 
where he was released by police after being detained for 24 hours.
hypothesis: 'Bakri was briefly detained, but was released.
relation: """

In [ ]:
response = openai.Completion.create(engine='text-davinci-001', prompt=p, max_tokens = 60)

In [ ]:
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nOmar Bakri was barred from returning to Britain from Lebanon."
    }
  ],
  "created": 1666716652,
  "id": "cmpl-65HQa9f1rN7gom46LXuMdIQVSeDJq",
  "model": "text-davinci-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 63,
    "total_tokens": 78
  }
}


In [ ]:
# query gpt with a prompt
prompt = """ Decide whether the sentence's sentiment is positive or negative
sentence: I hate this movie!
sentiment:"""

In [ ]:
# query gpt with a prompt
prompt = """ Decide whether the relation between premise and hypothesis have entailment or contradiction
premise: No Weapons of Mass Destruction Found in Iraq Yet.
hypothesis: Weapons of Mass Destruction Found in Iraq.
relation: not entailment

premise: 'Britain said, Friday, that it has barred cleric, Omar Bakri, from returning to the country from Lebanon, 
where he was released by police after being detained for 24 hours.
hypothesis: 'Bakri was briefly detained, but was released.
relation: 

"""

In [ ]:
response = openai.Completion.create(engine='text-davinci-001', prompt=prompt, max_tokens = 60)
#response = openai.Completion.create(engine='text-ada-001', prompt=prompt, max_tokens = 6)

In [ ]:
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nnot entailment"
    }
  ],
  "created": 1666716077,
  "id": "cmpl-65HHJgpmyghC6mc6SHe3xKbBJ0nyu",
  "model": "text-davinci-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 4,
    "prompt_tokens": 113,
    "total_tokens": 117
  }
}


In [ ]:
response["choices"][0]["text"].strip('\n')

'not entailment'

In [ ]:
# get rte dataset from huggingface
!pip install datasets

In [ ]:
import datasets as datasets

In [ ]:
# RTE dataset from Super GLUE - https://paperswithcode.com/sota/natural-language-inference-on-rte
# 0 - enatilment
# 1 - not entailment
dataset = 'rte'

In [ ]:
if dataset == 'rte':
  train_set = datasets.load_dataset('super_glue',dataset, split='train')
  dev_set = datasets.load_dataset('super_glue', dataset, split='validation')

In [ ]:
train_set.shape

(2490, 4)

In [ ]:
dev_set.shape

(277, 4)

In [ ]:
train_set[5]

{'premise': 'Britain said, Friday, that it has barred cleric, Omar Bakri, from returning to the country from Lebanon, where he was released by police after being detained for 24 hours.',
 'hypothesis': 'Bakri was briefly detained, but was released.',
 'idx': 5,
 'label': 0}

In [ ]:
# create a new toy dataset with very few examples from train dataset
new_toy_set = {}

In [ ]:
for i in range(0,5):
  new_toy_set[i] = train_set[i]

In [ ]:
new_toy_set

{0: {'premise': 'No Weapons of Mass Destruction Found in Iraq Yet.',
  'hypothesis': 'Weapons of Mass Destruction Found in Iraq.',
  'idx': 0,
  'label': 1},
 1: {'premise': 'A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI.',
  'hypothesis': 'Pope Benedict XVI is the new leader of the Roman Catholic Church.',
  'idx': 1,
  'label': 0},
 2: {'premise': 'Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.',
  'hypothesis': 'Herceptin can be used to treat breast cancer.',
  'idx': 2,
  'label': 0},
 3: {'premise': 'Judie Vivian, chief executive at ProMedica, a medical service company that helps sustain the 2-year-old Vietnam Heart Institute in Ho Chi Minh City (formerly Saigon), said that 

In [ ]:
for i in range(0,5):
  new_toy_set[i]['template'] = ' Given hypothesis is this the right prediction? '

In [ ]:
new_toy_set

{0: {'premise': 'No Weapons of Mass Destruction Found in Iraq Yet.',
  'hypothesis': 'Weapons of Mass Destruction Found in Iraq.',
  'idx': 0,
  'label': 1,
  'template': ' Given hypothesis is this the right prediction? '},
 1: {'premise': 'A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI.',
  'hypothesis': 'Pope Benedict XVI is the new leader of the Roman Catholic Church.',
  'idx': 1,
  'label': 0,
  'template': ' Given hypothesis is this the right prediction? '},
 2: {'premise': 'Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.',
  'hypothesis': 'Herceptin can be used to treat breast cancer.',
  'idx': 2,
  'label': 0,
  'template': ' Given hypothesis is this the right prediction? 

In [ ]:
example = """ Decide whether the relation between premise and hypothesis have entailment or not entailment
premise: No Weapons of Mass Destruction Found in Iraq Yet.
hypothesis: Weapons of Mass Destruction Found in Iraq.
relation: not entailment"""

In [ ]:
gpt_3_prompts = []

for i in range(0,5):
  gpt_3_prompts.append(example + "premise:" + 
                       new_toy_set[i]['premise']  + '\n' + 'hypothsis:' + 
                       new_toy_set[i]['hypothesis'] + "\n" + "relation: ")

In [ ]:
gpt_3_prompts

[' Decide whether the relation between premise and hypothesis have entailment or not entailment\npremise: No Weapons of Mass Destruction Found in Iraq Yet.\nhypothesis: Weapons of Mass Destruction Found in Iraq.\nrelation: not entailmentpremise:No Weapons of Mass Destruction Found in Iraq Yet.\nhypothsis:Weapons of Mass Destruction Found in Iraq.\nrelation: ',
 ' Decide whether the relation between premise and hypothesis have entailment or not entailment\npremise: No Weapons of Mass Destruction Found in Iraq Yet.\nhypothesis: Weapons of Mass Destruction Found in Iraq.\nrelation: not entailmentpremise:A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI.\nhypothsis:Pope Benedict XVI is the new leader of the Roman Catholic Church.\nrelation: ',
 ' Decide whether the relation between premise and hypothesis have entailment or not entailment\npremise: No Weapo

In [ ]:
gpt_results = []
for prompt in gpt_3_prompts:
  # prompt gpt-3 with the input and get the output
  response = openai.Completion.create(engine='text-davinci-001', prompt=prompt, max_tokens = 5, temperature=0)
  # extract the text from the rest of the data
  gpt_results.append(response["choices"][0]["text"])

In [ ]:
results_text = [result.strip('\n') for result in gpt_results]
results_num = [0 if result == 'entailment' else 1 for result in results_text]

In [ ]:
print(results_text)

['entailment', 'entailment', 'entailment', 'entailment', 'entailment']


In [ ]:
print(results_num)

[0, 0, 0, 0, 0]


In [ ]:
# actual results from rte dataset
true_label = [new_toy_set[sample]['label'] for sample in new_toy_set]
print(true_label)

[1, 0, 0, 0, 1]
